# **Air Quality Monitor Representativeness in the Contiguous United States**

#### **Objective:** Determine whether state and local governments site air monitors in areas that capture data that are representative of actual air quality.

#### **Scope:**

#### **Sources:**

| Num. | Title | Description | Source Link |
| :--- | :------------------------------ | :-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 1. | _daily\_88101\_2019.csv_ | Derived from the U.S. EPA Pre\-Generated Data Files webpage on 10/28/2025, this dataset contains the PM2.5 air monitor data for every day in the year 2019 for all monitors in the United States and its territories;                                                                                                    \-\- Data Dictionary: [https://www.epa.gov/aqs/aqs\-data\-dictionary](https://www.epa.gov/aqs/aqs-data-dictionary) | [https://aqs.epa.gov/aqsweb/airdata/download\_files.html\#Daily](https://aqs.epa.gov/aqsweb/airdata/download_files.html#Daily) \(see "Particulates" and "2019"\) |
| 2. | USHAP\_PM2.5\_D1K\_2019\_V1.zip | Derived from research out of the Universities of Maryland and Iowa on 11/03/2025, this dataset contains 1\-kilometer resolution PM2.5 estimates in the contiguous United States for every day in the year 2019. The dataset was generated from big data \(e.g., ground\-based measurements, satellite remote sensing products, atmospheric reanalysis, and model simulations\) using artificial intelligence. The researchers show their estimates align well with physical measurements, with 0.82 cross\-validated coefficients of determination. | Paper: [https://www.thelancet.com/journals/lanplh/article/PIIS2542\-5196\(23\)00235\-8/fulltext](https://www.thelancet.com/journals/lanplh/article/PIIS2542-5196(23)00235-8/fulltext) <br> <br>  Data: <br> https://zenodo.org/records/7884640 |



#### Methodology

1. AQS Data Cleaning
   1. Download AQS Dataset
   2. Load Libraries
   3. Load AQS Dataset
   4. Select Relevant AQS Dataset Variables
   5. Generate a Dataset of PM2.5 Monitors with Locations
2. Alternative PM2.5 Values Dataset
   1. 



### **AQS Data Cleaning**



#### AQS Data Cleaning Pseudocode

1. Load libraries I will likely need
2. Load AQS dataset: DF &lt;\- read.csv
3. Limited\_DF &lt;\- Select\(State.Code, County.Code, Site.Num, Latitude, Longitude, Address, [State.Name](http://State.Name), [County.Name](http://County.Name), [City.Name](http://City.Name)\)
4. Unique identifier \(AQS\_ID\) &lt;\- Leading\_Zeros\_If\_Needed\(State.Code \+ County.Code  \+ Site.Num\)
5. Filter to unique AQS\_ID w/ locations



#### Load Libraries



In [13]:
#Purpose of Cell Block: Load libraries I will likely need
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(readr))
suppressPackageStartupMessages(library(vroom))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(terra))

if (!require(dplyr)) install.packages("dplyr")
if (!require(dplyr)) install.packages("tidyr")
if (!require(readr)) install.packages("readr")
if (!require(vroom)) install.packages("vroom")
if (!require(ggplot2)) install.packages("ggplot2")
if (!require(stringr)) install.packages("stringr")
if (!require(stringr)) install.packages("terra")

library(dplyr)
library(tidyr)
library(readr)
library(vroom)
library(ggplot2)
library(stringr)
library(terra)

#### Load AQS Dataset



In [14]:
#Purpose of Cell Block: Get dataset
DF_aqs2019 <- read.csv("data/daily_88101_2019.csv")

In [15]:
#Purpose of Cell Block: Explore uploaded dataset

#head(DF_aqs2019)
dim(DF_aqs2019)
names(DF_aqs2019)
#glimpse(DF_aqs2019)
#summary(DF_aqs2019)

#Assessment: The dataset is too large. Need to select only the variables of interest. It also appears the AQS ID variables dropped their leading zeros. I will need to fix that issue for a proper unique identifier 

[1] 654337     29

[1] "State.Code"          "County.Code"         "Site.Num"           
 [4] "Parameter.Code"      "POC"                 "Latitude"           
 [7] "Longitude"           "Datum"               "Parameter.Name"     
[10] "Sample.Duration"     "Pollutant.Standard"  "Date.Local"         
[13] "Units.of.Measure"    "Event.Type"          "Observation.Count"  
[16] "Observation.Percent" "Arithmetic.Mean"     "X1st.Max.Value"     
[19] "X1st.Max.Hour"       "AQI"                 "Method.Code"        
[22] "Method.Name"         "Local.Site.Name"     "Address"            
[25] "State.Name"          "County.Name"         "City.Name"          
[28] "CBSA.Name"           "Date.of.Last.Change"

#### Select Relevant AQS Dataset Variables



In [16]:
#Purpose of Cell Block: Pulling relevant variables for a smaller dataset

DF_aqs2019_loc <- DF_aqs2019 %>% 
    select('State.Code':'Site.Num','Latitude','Longitude','Address':'City.Name') %>% 
    rename('State' = 'State.Name', 'County' = 'County.Name', 'City' = 'City.Name')

#Confirming new dataset 
head(DF_aqs2019_loc)
dim(DF_aqs2019_loc)
names(DF_aqs2019_loc)
glimpse(DF_aqs2019_loc)
summary(DF_aqs2019_loc)

,State.Code,County.Code,Site.Num,Latitude,Longitude,Address,State,County,City
,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,1,3,10,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope
2,1,3,10,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope
3,1,3,10,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope
4,1,3,10,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope
5,1,3,10,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope
6,1,3,10,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope


[1] 654337      9

[1] "State.Code"  "County.Code" "Site.Num"    "Latitude"    "Longitude"  
[6] "Address"     "State"       "County"      "City"

Rows: 654,337
Columns: 9
$ State.Code  <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ County.Code <int> 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3…
$ Site.Num    <int> 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10…
$ Latitude    <dbl> 30.49748, 30.49748, 30.49748, 30.49748, 30.49748, 30.49748…
$ Longitude   <dbl> -87.88026, -87.88026, -87.88026, -87.88026, -87.88026, -87…
$ Address     <chr> "FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE,  ALABAMA"…
$ State       <chr> "Alabama", "Alabama", "Alabama", "Alabama", "Alabama", "Al…
$ County      <chr> "Baldwin", "Baldwin", "Baldwin", "Baldwin", "Baldwin", "Ba…
$ City        <chr> "Fairhope", "Fairhope", "Fairhope", "Fairhope", "Fairhope"…


   State.Code     County.Code        Site.Num       Latitude    
 Min.   : 1.00   Min.   :  1.00   Min.   :   1   Min.   :17.71  
 1st Qu.:13.00   1st Qu.: 19.00   1st Qu.:   7   1st Qu.:35.32  
 Median :29.00   Median : 51.00   Median :  24   Median :39.41  
 Mean   :28.54   Mean   : 73.56   Mean   : 691   Mean   :38.52  
 3rd Qu.:42.00   3rd Qu.:103.00   3rd Qu.:1001   3rd Qu.:41.77  
 Max.   :78.00   Max.   :810.00   Max.   :9997   Max.   :64.85  
   Longitude         Address             State              County         
 Min.   :-159.37   Length:654337      Length:654337      Length:654337     
 1st Qu.:-109.43   Class :character   Class :character   Class :character  
 Median : -89.87   Mode  :character   Mode  :character   Mode  :character  
 Mean   : -94.93                                                           
 3rd Qu.: -80.87                                                           
 Max.   : -64.78                                                           
     City    

##### **Site ID Format**

**State Code**

A two\-digit FIPS code that identifies one of the 50 US states.  May also indicate a Tribal Area \(TT\) or other country such as Canada \(CC\) or Mexico \(80\).

**County Code**

A three\-digit FIPS code that identifies a county or the equivalent geo\-political entity, such as a parish or independent city.

**Tribal Code**

Only visible in Tribal Mode.  A two\-digit FIPS code that identifies one of the federally recognized Native American tribal agencies when the Tribal Ind is set to "TT".  If your Screening Group also has non\-tribal data, this column will contain the County Code.

**Site Number**

A four\-digit number used to uniquely identify a site within a county.  Defined and maintained by the states or Tribal agencies.  When this is a Tribal site care must be taken to ensure that the site ID is unique not only in the Tribal Area but also in all the counties that the tribal area includes or crosses.

Source: [https://www.epa.gov/aqs/site\-id\-format](https://www.epa.gov/aqs/site-id-format) 


In [17]:
#Purpose of Cell Block: I will need a single identifier for each monitor, meaning I need to combine the identifier variables. The variables need to match the format shown in the markdown cell block above, meaning I will first need to add leading zeros to the data if needed

DF_aqs2019_loc2 <- DF_aqs2019_loc %>%
    mutate('State.Code.Fix' = str_pad(State.Code, width = 2, pad = 0)) %>% #pad leading zero so state code always 2 char
    mutate('County.Code.Fix' = str_pad(County.Code, width = 3, pad = 0)) %>% #pad leading zero so county code always 3 char
    mutate('Site.Num.Fix' = str_pad(Site.Num, width = 4, pad = 0)) %>% #pad leading zero so site num always 4 char
    select(-('State.Code':'Site.Num')) %>%  #drop old codes
    unite(
        col = 'AQS_Site_ID','State.Code.Fix', 
        'County.Code.Fix', 
        'Site.Num.Fix', 
        sep = "", 
        remove = TRUE
    )

head(DF_aqs2019_loc2)

#QA check to make sure Site ID len = 9
DF_LenCheck <- DF_aqs2019_loc2 %>%
    mutate(LenCheck = nchar(AQS_Site_ID)) %>% #add column to count the number of characters in AQS_Site_ID
    filter(!LenCheck %in% c(9)) #filter to only show rows with different than 9 characters
    
head(DF_LenCheck)
#Conclusion: No AQS_Site_ID values contain more or less than 9 characters

,Latitude,Longitude,Address,State,County,City,AQS_Site_ID
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope,010030010
2,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope,010030010
3,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope,010030010
4,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope,010030010
5,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope,010030010
6,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope,010030010


Latitude,Longitude,Address,State,County,City,AQS_Site_ID,LenCheck
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>


#### Generate a Dataset of PM2.5 Monitors with Locations



In [18]:
#Purpose of Cell Block: Reduce the dataset to only unique AQS_Site_ID (dataset was originally "daily observations")
DF_aqs2019_loc3 <- DF_aqs2019_loc2 %>%
    distinct()  %>%
    relocate(AQS_Site_ID, Latitude, Longitude, Address, State, County, City) #cleaning up dataframe column order

#Check total number of monitors
Monitors2019 <- DF_aqs2019_loc3 %>%
    summarise(total_sites = n_distinct(AQS_Site_ID))

print(paste0(Monitors2019$total_sites, " PM2.5 monitors operating in 2019"))
head(DF_aqs2019_loc3)
#Conclusion: 996 PM2.5 monitors operated in the United States and its territories in 2019

[1] "996 PM2.5 monitors operating in 2019"


,AQS_Site_ID,Latitude,Longitude,Address,State,County,City
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,010030010,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope
2,010270001,33.28493,-85.80361,ASHLAND AIRPORT,Alabama,Clay,Ashland
3,010331002,34.76262,-87.63810,WILSON DAM RD AND 2ND ST.,Alabama,Colbert,Muscle Shoals
4,010491003,34.28857,-85.96986,"13112 HWY.68 , CROSSVILLE AL. 35962",Alabama,DeKalb,Crossville
5,010550010,33.98821,-85.99256,"314 COLLEGE DRIVE, GADSDEN, AL 35902",Alabama,Etowah,Gadsden
6,010690003,31.22478,-85.39079,126 NORTH ST. ANDREWS STREET (CIVIC CENTER),Alabama,Houston,Dothan


### **Alternative PM2.5 Dataset**



#### Alternative PM2.5 Data Cleaning Pseudocode

1. Load the 364 Alternative PM2.5 datasets: DF &lt;\- read.netcdf
2. Append 364 datasets into a single dataframe
3. Note we need to add our values to the air monitor dataset so we can assess how well they relate to physical air measurements. we take this step prior to using the data to identify hotspots for each county



In [19]:
#Purpose of Cell Block: Locate and ready Alt. PM2.5 datasets 

altPM25_files <- list.files("data", pattern = "\\.nc$", full.names = TRUE) #identify the number of alt. PM2.5 files in data folder
n_distinct(altPM25_files) #count alt. PM2.5 files
#altPM25_files

[1] 364

In [20]:
#Purpose of Cell Block: Grab Alt. PM2.5 value for each monitor location on each day

AltPM25_filename_dates <- function(f) {
    
    
}


AQS_Site_ID,Latitude,Longitude,Address,State,County,City
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
010030010,30.49748,-87.88026,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",Alabama,Baldwin,Fairhope
010270001,33.28493,-85.80361,ASHLAND AIRPORT,Alabama,Clay,Ashland
010331002,34.76262,-87.63810,WILSON DAM RD AND 2ND ST.,Alabama,Colbert,Muscle Shoals
010491003,34.28857,-85.96986,"13112 HWY.68 , CROSSVILLE AL. 35962",Alabama,DeKalb,Crossville
010550010,33.98821,-85.99256,"314 COLLEGE DRIVE, GADSDEN, AL 35902",Alabama,Etowah,Gadsden
010690003,31.22478,-85.39079,126 NORTH ST. ANDREWS STREET (CIVIC CENTER),Alabama,Houston,Dothan
010730023,33.55306,-86.81500,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham
010731005,33.33111,-87.00361,ROUTE 8 MCADORY,Alabama,Jefferson,Not in a city
010731010,33.54528,-86.54917,201 ASHVILLE ROAD,Alabama,Jefferson,Leeds
